##Idea based on: A neural interlingua for multilingual machine translation.

## Basic idea
* Implement a multilingual encoder-interlingua-decoder model and train it on three languages, for example en, sv and de.

* Train model on translating the following language pairs:
    * en - de
    * de - sv
    * en - en
    * sv - sv
    * de - de.

* Train a bilingual model only consisting of a encoder-decoder pair and train it on the language pair: en-sv. This will be used for reference when evaluating the model.

* Evaluate the multilingual model by letting the first model zero-shot translate en-sv, then compare results with the bilingual model. Hopefully they wont be to far apart. 

###Possible improvements
* Use Bert embedding, this would make the model larger, but might yield a higher score as we have seen previously. I belive this is a bit harder to implement and thus I wont start of with it.

* Include more languages in the interlingua model. This will make the training take more time but how it will affect the results I find unclear. Three languages is the bare minimum for a zero-shot translation to be possible. 

###Implementation
* Use mt_util.py for dataloading as in machine translation colab notebook.
* Use code from the same notebook for the bilingual model with BahdanauAttention.
* Modify the bilingual model to include a second interlingua attention.
* Write a script that trains the multilingual model. 


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Install the library to compute the BLEU score.
!pip install sacrebleu

# Get the Python file that contains the auxiliary functions for this exercise.
!wget https://raw.githubusercontent.com/liu-nlp/dl4nlp/master/exercise3_1/mt_util.py

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import time
import numpy as np
import sys
import random
import matplotlib.pyplot as plt
import sacrebleu
from collections import defaultdict

import mt_util

     |████████████████████████████████| 90 kB 4.2 MB/s 
--2022-01-10 12:29:56--  https://raw.githubusercontent.com/liu-nlp/dl4nlp/master/exercise3_1/mt_util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7686 (7.5K) [text/plain]
Saving to: ‘mt_util.py’

mt_util.py          100%[===================>]   7.51K  --.-KB/s    in 0s      

2022-01-10 12:29:56 (49.1 MB/s) - ‘mt_util.py’ saved [7686/7686]



# Bilingual model
Using code from week 5 machine translation notebook and tweaking it a bit to fit our specific purpose.

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    # S is a batch of source-language sentences, shape (n_sentences, n_source_words).
    # T_shifted contains the corresponding target-language sentences, shifted one step
    #   to the right for teacher forcing, shape (n_sentences, n_target_words).
    def forward(self, S, T_shifted):        
        # Apply the encoder, we get token representations in the source language.
        S_enc = self.encoder(S)

        # Compute a mask that we'll use to get rid of padding tokens when we compute
        # the attention. (We hardcode that the padding token has index 0.)
        S_mask = (S != 0)
        
        # Apply the decoder in teacher forcing mode.
        return self.decoder(S_enc, S_mask, T_shifted)
    
    # S is a batch of source-language sentences, shape (n_sentences, n_source_words).
    # max_len is the maximally allowed sentence length.
    def greedy_decode(self, S, max_len):
        S_enc = self.encoder(S)    
        S_mask = (S != 0)
        return self.decoder.greedy_decode(S_enc, S_mask, max_len)

class Encoder(nn.Module):
    def __init__(self, emb_layer, param):
        super().__init__()
        _, emb_dim = emb_layer.weight.shape       
        self.emb_layer = emb_layer
        self.rnn = nn.GRU(emb_dim, param.enc_rnn_dim, param.enc_rnn_depth, 
                          batch_first=True, bidirectional=True)
        
    def forward(self, S):
        embedded = self.emb_layer(S)
        output, _ = self.rnn(embedded)
        return output

class Decoder(nn.Module):

    def __init__(self, emb_layer, param):
        super().__init__()
        voc_size, emb_dim = emb_layer.weight.shape

        # Word embeddings for the target language. 
        self.emb_layer = emb_layer

        attn_dim = 2*param.enc_rnn_dim

        # The decoder's state is represented using a GRU, called f in Bahdanau's paper.
        # The input of this GRU is a combination of the embedding of the previous word
        # and the "context" computed by the attention model.
        self.rnn = nn.GRU(emb_dim + attn_dim, param.dec_rnn_dim, param.dec_rnn_depth, 
                          batch_first=True, bidirectional=False)
        
        # The output layer (called g in the paper) is a bit simpler than in the paper. 
        # We just use a linear model.
        self.output_layer = nn.Linear(param.dec_rnn_dim + attn_dim + emb_dim,
                                      voc_size, bias=False)
                
        self.rnn_dim = param.dec_rnn_dim

        # The attention model, which computes a context or "summary" of the source sentence.
        # Right now, we use a simplified attention model that doesn't depend on the decoder state.
        # See below for the implementation.
        # self.attention = MeanAttention()
        self.attention = BahdanauAttention(param.enc_rnn_dim*2, param.enc_rnn_dim, param.hidden_dim)

        
    # This method applies the attention model, the RNN and the output model at one
    # position in the decoded sentence.
    def forward_step(self, prev_embed, enc_out, src_mask, precomputed_key, decoder_hidden):        

        # The "query" for attention is the hidden state of the decoder.
        query = decoder_hidden[-1].unsqueeze(1)  
        
        # Apply the attention model to compute a "context", summary of the encoder output
        # based on the current query.
        # Also returns the attention weights, which we might want to visualize or inspect.
        context, attn_probs = self.attention(
            query=query, precomputed_key=precomputed_key,
            value=enc_out, mask=src_mask)
        
        # Run the RNN one step.
        rnn_input = torch.cat([prev_embed, context], dim=2)
        rnn_output, decoder_hidden = self.rnn(rnn_input, decoder_hidden)
        
        # The word output model (called g in Bahdanau's paper) uses the embedding 
        # of the previous word, the RNN output, and the context computed by the attention model.
        pre_output = torch.cat([prev_embed, rnn_output, context], dim=2)
        T_output = self.output_layer(pre_output)

        return attn_probs, decoder_hidden, T_output
        
        
    # Apply the decoder in teacher forcing mode.
    def forward(self, enc_out, src_mask, T_shifted):

        n_sen, n_words = T_shifted.shape
        
        # Word embeddings for the shifted word, representing the previous step.
        T_emb = self.emb_layer(T_shifted)
        
        # Precompute attention keys (if needed).
        precomputed_key = self.attention.precompute_key(enc_out)
        
        # Initialize the hidden state of the GRU.
        decoder_hidden = torch.zeros(1, n_sen, self.rnn_dim, device=src_mask.device)
        
        all_out = []
        
        # For each position in the target sentence:
        for i in range(n_words):
            
            # Embedding for the previous word.
            prev_embed = T_emb[:, i].unsqueeze(1)
            
            # Run the decoder one step.
            # This returns a new hidden state, and the output 
            # scores (over the target vocabulary) at this position.
            _, decoder_hidden, T_output = self.forward_step(prev_embed, enc_out, 
                                                            src_mask, precomputed_key, decoder_hidden)
            all_out.append(T_output)
 
        # Combine the output scores for all positions.
        return torch.cat(all_out, dim=1)
            
    # Apply the decoder in a greedy step-by-step fashion, at each step selecting the
    # highest-scoring word.
    def greedy_decode(self, enc_out, src_mask, max_len):
        n_sen, _ = src_mask.shape

        precomputed_key = self.attention.precompute_key(enc_out)
        decoder_hidden = torch.zeros(1, n_sen, self.rnn_dim, device=src_mask.device)
        
        prev_tokens = torch.zeros(src_mask.size(0), 1, dtype=torch.long, 
                                  device=src_mask.device)
        all_out = []
        all_attn = []

        for i in range(max_len):
            prev_embed = self.emb_layer(prev_tokens)
            attn_probs, decoder_hidden, T_output = self.forward_step(prev_embed, enc_out, src_mask, 
                                                                     precomputed_key, decoder_hidden)
            # Select the highest-scoring word.
            prev_tokens = T_output.argmax(-1)
            all_out.append(prev_tokens)
            all_attn.append(attn_probs)
        
        return torch.cat(all_out, dim=1), torch.stack(all_attn)



## Bahdanau attention
For opur attention mechanism we will use the one defined in the paper by Bahdanau et al.

In [ ]:
class BahdanauAttention(nn.Module):
    
    def __init__(self, input_dim, query_dim, hidden_dim):
        super().__init__()
        self.K = nn.Linear(input_dim, hidden_dim)
        self.Q = nn.Linear(query_dim, hidden_dim)
        self.v_a = nn.Linear(hidden_dim, 1)
        self.f = nn.Tanh() # Activation function
                   
    # When we implement the full attention mechanisms, we'll want to precompute the "key"
    # representations. For now, we do nothing here.
    def precompute_key(self, encoder_out):
        return self.K(encoder_out)
            
    def forward(self, query, precomputed_key, value, mask):
        # n_sen, n_words, enc_rnn_dim = value.shape
        
        Qs = self.Q(query)
        summarized = torch.add(Qs, precomputed_key)
        scores = self.v_a( self.f(summarized) )
        scores = torch.squeeze(scores, dim=2)

        # Mask out the energy scores for the padding tokens.
        # We set them to -infinity so that they will be 0 after applying the softmax.
        if mask != None:
            scores.data.masked_fill_(~mask, -float('inf'))
                
        # Turn scores to probabilities.
        alphas = F.softmax(scores, dim=1)        
        
        # The context vector is the weighted sum of the values.
        context = torch.bmm(alphas.unsqueeze(1), value)
        
        return context, alphas


## Parameters and training loop

In [ ]:
class TranslatorParameters:
    device = 'cuda'
    
    n_batches_print = 32
    
    src_voc_size = 10000
    tgt_voc_size = 10000
    
    random_seed = 0
            
    n_epochs = 37
    
    batch_size = 64
    
    learning_rate = 2e-4
    weight_decay = 0
        
    emb_dim = 128
    
    enc_rnn_dim = 128 
    enc_rnn_depth = 1

    dec_rnn_dim = 128 
    dec_rnn_depth = 1 

    hidden_dim = 64

    max_train_sentences = 200000
    max_valid_sentences = 1000
    
    src_train = './data/de-en-train.de'
    tgt_train = './data/de-en-train.en'
    src_valid = './data/de-en-test.de'
    tgt_valid = './data/de-en-test.en'
      
    
class Translator:
    
    def __init__(self, params):
        self.params = params
        
    def train(self):
        
        p = self.params
        
        # Setting a fixed seed for reproducibility.
        torch.manual_seed(p.random_seed)
        random.seed(p.random_seed)
        
        print('Preparing data...', end='')
        sys.stdout.flush()
        
        # Read the source-language data.
        S_train = mt_util.read_lines(p.src_train, max_lines=p.max_train_sentences)
        self.S_voc = mt_util.Vocabulary(include_unknown=True, lower=True, max_voc_size=p.src_voc_size)
        self.S_voc.build(S_train)
        S_valid = mt_util.read_lines(p.src_valid, max_lines=p.max_valid_sentences)

        # Read the target-language data.
        T_train = mt_util.read_lines(p.tgt_train, max_lines=p.max_train_sentences)
        self.T_voc = mt_util.Vocabulary(include_unknown=True, lower=True, max_voc_size=p.tgt_voc_size)
        self.T_voc.build(T_train)
        T_valid = mt_util.read_lines(p.tgt_valid, max_lines=p.max_valid_sentences)
                
        # Batching for the training and validation sets.
        self.batcher = mt_util.SequenceBatcher(p.device)        
        train_dataset = mt_util.SequenceDataset(self.S_voc.encode(S_train), self.T_voc.encode(T_train))
        train_loader = DataLoader(train_dataset, p.batch_size, shuffle=True, collate_fn=self.batcher)

        valid_dataset = mt_util.SequenceDataset(self.S_voc.encode(S_valid), self.T_voc.encode(T_valid))
        valid_loader = DataLoader(valid_dataset, p.batch_size, shuffle=True, collate_fn=self.batcher)
        
        print(' done.')
        
        print('Building model...', end='')
        sys.stdout.flush()        
        
        # We use a utility to build the embedding layer, if we would like to 
        # use a pre-trained model (which we don't do here).
        S_emb = self.S_voc.make_embedding_layer(finetune=True, emb_dim=p.emb_dim)
        T_emb = self.T_voc.make_embedding_layer(finetune=True, emb_dim=p.emb_dim)
        
        # Build the encoder and decoder.
        encoder = Encoder(S_emb, p)
        decoder = Decoder(T_emb, p)
        self.model = EncoderDecoder(encoder, decoder)
        
        self.model.to(p.device)
        print(' done.')
                
        optimizer = torch.optim.Adam(self.model.parameters(), 
                                     lr=p.learning_rate, weight_decay=p.weight_decay)

        # The loss function is a cross-entropy loss at the token level.
        # We don't include padding tokens when computing the loss.
        loss_func = torch.nn.CrossEntropyLoss(ignore_index=self.T_voc.get_pad_idx())

        self.history = defaultdict(list)
        for epoch in range(1, p.n_epochs+1):
            
            t0 = time.time()

            loss_sum = 0
            for i, (Sbatch, Tbatch) in enumerate(train_loader, 1):
                
                # We use teacher forcing to train the decoder.
                # This means that the input at each decoding step will be the
                # *gold-standard* word at the previous position.
                # We create a tensor Tbatch_shifted that contains the previous words.                
                batch_size, sen_len = Tbatch.shape
                zero_pad = torch.zeros(batch_size, 1, dtype=torch.long, device=Tbatch.device)
                Tbatch_shifted = torch.cat([zero_pad, Tbatch[:, :-1]], dim=1)
                
                self.model.train()
                scores = self.model(Sbatch, Tbatch_shifted)

                loss = loss_func(scores.view(-1, len(self.T_voc)), Tbatch.view(-1))

                optimizer.zero_grad()            
                loss.backward()
                optimizer.step()
                loss_sum += loss.item()

                print('.', end='')
                sys.stdout.flush()
                
                # We periodically print some diagnostics: loss, BLEU score on the validation
                # set, and the translation of a test sentence.
                if i % p.n_batches_print == 0:
                    # test_out = ' '.join(self.translate([p.test_sen.split()])[0])
                    # bleu = self.eval_bleu(S_valid, T_valid)
                    print(f' {i*p.batch_size}')#: loss={loss_sum/i:.4f} BLEU={bleu:.4f} | {p.test_sen} -> {test_out}')
                    
            print()
            t1 = time.time()
            
            train_loss = loss_sum / len(train_loader)
                                    
            print(f'Epoch {epoch}: train loss = {train_loss:.4f}, time = {t1-t0:.4f}')

            bleu = self.eval_bleu(S_valid, T_valid)
            self.history['bleu'].append(bleu)

            # Plot history
            plt.figure(figsize=(6, 5))
            plt.style.use('seaborn')
            x = range(1, epoch+1)
            # lines
            plt.plot(x, self.history['bleu'])
            # text
            plt.suptitle('Bilingual model', fontsize=20)
            plt.ylabel('BLEU', fontsize=14)
            plt.xlabel('epoch', fontsize=14)
            # save
            plt.savefig('fig1.png', dpi=150)
            plt.clf()





    # Utility function that runs the encoder and decoder for a batch.
    # We then map the target-language word indices to the corresponding strings,
    # and cut off the sentences after the end-of-sentence marker.
    # For visualization, we'll also return all the attention scores.
    def translate_batch(self, src):
        with torch.no_grad():
            encoded = self.S_voc.encode(src)
            max_len = max(len(s) for s in encoded)
            pad = self.S_voc.get_pad_idx()
            for s in encoded:
                s.extend([pad]*(max_len-len(s)))
            enc_tensor = torch.tensor(encoded, device=self.params.device)
            tgt, attn = self.model.greedy_decode(enc_tensor, max_len=2*max_len)
            tgt_dec = self.T_voc.decode(tgt[:,1:].cpu().numpy())
            for s in tgt_dec:
                try:
                    eos_ix = s.index(mt_util.EOS)
                    del s[eos_ix:]
                except:
                    pass
            return tgt_dec, attn.cpu().numpy()
        
    # Translates a set of sentences in the source language.
    def translate(self, src, batch_size=512):
        self.model.eval()
        out = []
        batch = []
        for sen in src:
            batch.append(sen)
            if len(batch) == batch_size:
                translated_batch, _ = self.translate_batch(batch)
                out.extend(translated_batch)
                batch = []
        if len(batch) > 0:
            translated_batch, _ = self.translate_batch(batch)
            out.extend(translated_batch)
        return out
         
    # Translates a single sentence and returns the translation and attention scores.
    def translate_with_attention(self, sentence):
        translated, attn = self.translate_batch([sentence])
        attn = attn.squeeze(1)
        return translated[0], attn
        
    # Runs the translator, and then uses the SacreBLEU evaluator to compute a BLEU score.
    def eval_bleu(self, src, ref):
        translated = self.translate(src)
        translated = [' '.join(s) for s in translated]
        ref = [' '.join(s).lower() for s in ref]
        return sacrebleu.raw_corpus_bleu(translated, [ref], 0.01).score
        


In [ ]:
translator = Translator(TranslatorParameters())
translator.train()

## Multilingual model

In [ ]:
class MultilingualModel(nn.Module):
    """
    Most of the code in this model is from the week 5 notebook on neural
    machine translation. A mayor difference is that a interlingua representation 
    has been added between the encoder and decoder, this implementation was done
    by following the model desciption described in Yichao Lu et al., however 
    we are using GRU instead of LSTM.
    """
    def __init__(self, encoder, interlingua, decoder):
        super().__init__()
        self.encoder = encoder
        self.interlingua = interlingua
        self.decoder = decoder
        
    def forward(self, S, T_shifted):        
        """ a normal forward pass with teacher enforcing """
        # encoding the data
        S_enc = self.encoder(S)

        # mask that contains information on the positions of the padding tokens
        S_mask = (S != 0)

        # computing interlingua representation
        S_interlingua = self.interlingua(S_enc, S_mask)

        # returning the decoded sentance as output
        return self.decoder(S_interlingua, S_mask, T_shifted)
    
    def greedy_decode(self, S, max_len):
        """ when predicting greedily we will not use teacher enforcing """
        S_enc = self.encoder(S)    
        S_mask = (S != 0)
        S_interlingua = self.interlingua(S_enc, S_mask)
        return self.decoder.greedy_decode(S_interlingua, S_mask, max_len)

class Encoder(nn.Module):
    def __init__(self, emb_layer, param):
        super().__init__()
        _, emb_dim = emb_layer.weight.shape       
        self.emb_layer = emb_layer
        self.rnn = nn.GRU(emb_dim, param.enc_rnn_dim, param.enc_rnn_depth, 
                          batch_first=True, bidirectional=True)
    def forward(self, S):
        embedded = self.emb_layer(S)
        output, _ = self.rnn(embedded)
        return output

class Interlingua(nn.Module):

    def __init__(self, param):
        super().__init__()

        self.rnn_dim = param.inter_rnn_dim
        self.max_sent_len = param.max_sent_len

        attn_dim = 2*param.enc_rnn_dim # twice the size since encoder is bidirectional
        self.rnn = nn.GRU(attn_dim, param.inter_rnn_dim, param.inter_rnn_depth,
                          batch_first=True, bidirectional=False)
        self.attention = BahdanauAttention(attn_dim, param.inter_rnn_dim, param.hidden_dim)
        self.output_layer = nn.Linear(param.inter_rnn_dim, param.inter_rnn_dim)


    def forward_step(self, inter_out, src_mask, precomputed_key, inter_hidden):        

        # The "query" for attention is the hidden state of the decoder.
        query = inter_hidden[-1].unsqueeze(1)  
        
        # Compute attention
        context, attn_probs = self.attention(
            query=query, precomputed_key=precomputed_key,
            value=inter_out, mask=src_mask)
        
        # Run the RNN one step.
        rnn_output, inter_hidden = self.rnn(context, inter_hidden)
        output = self.output_layer(rnn_output)
        
        return attn_probs, inter_hidden, output

    def forward(self, enc_out, src_mask):

        # n_sen, n_words = T_shifted.shape
        n_sen = enc_out.shape[0]
        
        # Precompute attention keys 
        precomputed_key = self.attention.precompute_key(enc_out)
        
        # Initialize the hidden state of the GRU.
        inter_hidden = torch.zeros(1, n_sen, self.rnn_dim, device=src_mask.device)
        
        all_out = []
        for _ in range(self.max_sent_len*2): # for each interlingua representation
            
            # Run the decoder one step.
            # This returns a new hidden state, and the output 
            # scores (over the target vocabulary) at this position.
            _, inter_hidden, output = self.forward_step(enc_out, src_mask, precomputed_key, inter_hidden)
            all_out.append(output)
 
        # Combine the output scores for all positions.
        return torch.cat(all_out, dim=1)
            


class Decoder(nn.Module):

    def __init__(self, emb_layer, param):
        super().__init__()
        voc_size, emb_dim = emb_layer.weight.shape

        # Word embeddings for the target language. 
        self.emb_layer = emb_layer

        self.rnn_dim = param.dec_rnn_dim

        # The decoder's state is represented using a GRU, called f in Bahdanau's paper.
        # The input of this GRU is a combination of the embedding of the previous word
        # and the "context" computed by the attention model.
        self.rnn = nn.GRU(emb_dim + param.dec_rnn_dim, param.dec_rnn_dim, param.dec_rnn_depth, 
                          batch_first=True, bidirectional=False)
        
        # The output layer (called g in the paper) is a bit simpler than in the paper. 
        # We just use a linear model.
        self.output_layer = nn.Linear(param.dec_rnn_dim * 2 + emb_dim,
                                      voc_size, bias=False)
                

        # The attention model, which computes a context or "summary" of the source sentence.
        # Right now, we use a simplified attention model that doesn't depend on the decoder state.
        # See below for the implementation.
        self.attention = BahdanauAttention(param.inter_rnn_dim, param.enc_rnn_dim, param.hidden_dim)

        
    # This method applies the attention model, the RNN and the output model at one
    # position in the decoded sentence.
    def forward_step(self, prev_embed, inter_out, precomputed_key, decoder_hidden):        

        # The "query" for attention is the hidden state of the decoder.
        query = decoder_hidden[-1].unsqueeze(1)  
        
        # Apply the attention model to compute a "context", summary of the encoder output
        # based on the current query.
        # Also returns the attention weights, which we might want to visualize or inspect.
        context, attn_probs = self.attention(
            query=query, precomputed_key=precomputed_key,
            value=inter_out, mask=None)
        
        # Run the RNN one step.
        rnn_input = torch.cat([prev_embed, context], dim=2)
        rnn_output, decoder_hidden = self.rnn(rnn_input, decoder_hidden)
        
        # The word output model (called g in Bahdanau's paper) uses the embedding 
        # of the previous word, the RNN output, and the context computed by the attention model.
        pre_output = torch.cat([prev_embed, rnn_output, context], dim=2)
        T_output = self.output_layer(pre_output)

        return attn_probs, decoder_hidden, T_output
        
        
    # Apply the decoder in teacher forcing mode.
    def forward(self, inter_out, src_mask, T_shifted):

        n_sen, n_words = T_shifted.shape
        
        # Word embeddings for the shifted word, representing the previous step.
        T_emb = self.emb_layer(T_shifted)
        
        # Precompute attention keys (if needed).
        precomputed_key = self.attention.precompute_key(inter_out)
        
        # Initialize the hidden state of the GRU.
        decoder_hidden = torch.zeros(1, n_sen, self.rnn_dim, device=src_mask.device)
        
        all_out = []
        
        # For each position in the target sentence:
        for i in range(n_words):
            
            # Embedding for the previous word.
            prev_embed = T_emb[:, i].unsqueeze(1)
            
            # Run the decoder one step.
            # This returns a new hidden state, and the output 
            # scores (over the target vocabulary) at this position.
            _, decoder_hidden, T_output = self.forward_step(prev_embed, inter_out, 
                                                            precomputed_key, decoder_hidden)
            all_out.append(T_output)
 
        # Combine the output scores for all positions.
        return torch.cat(all_out, dim=1)
            
    # Apply the decoder in a greedy step-by-step fashion, at each step selecting the
    # highest-scoring word.
    def greedy_decode(self, enc_out, src_mask, max_len):
        n_sen, _ = src_mask.shape

        precomputed_key = self.attention.precompute_key(enc_out)
        decoder_hidden = torch.zeros(1, n_sen, self.rnn_dim, device=src_mask.device)
        
        prev_tokens = torch.zeros(src_mask.size(0), 1, dtype=torch.long, 
                                  device=src_mask.device)
        all_out = []
        all_attn = []

        for i in range(max_len):
            prev_embed = self.emb_layer(prev_tokens)
            attn_probs, decoder_hidden, T_output = self.forward_step(prev_embed, enc_out, 
                                                                     precomputed_key, decoder_hidden)
            # Select the highest-scoring word.
            prev_tokens = T_output.argmax(-1)
            all_out.append(prev_tokens)
            all_attn.append(attn_probs)
        
        return torch.cat(all_out, dim=1), torch.stack(all_attn)



## Parameters and training loop

In [ ]:
class TranslatorParameters_milti:
    device = 'cuda'
    
    n_batches_print = 32
    
    src_voc_size = 10000
    tgt_voc_size = 10000
    
    random_seed = 0
            
    n_epochs = 100
    
    batch_size = 64

    hidden_dim = 64# hidden dim used when computing attention

    max_sent_len = 25
    
    learning_rate = 2e-4
    weight_decay = 0
        
    emb_dim = 128
    
    enc_rnn_dim = 128
    enc_rnn_depth = 2

    inter_rnn_dim = 128
    inter_rnn_depth = 1

    dec_rnn_dim = 128
    dec_rnn_depth = 1
    
    max_train_sentences = 200000
    max_train_sentences_identity = 10000
    max_valid_sentences = 1000

    n_languages = 3

    # Training datasets
    de_en = {'src_train': './data/de-en-train.de',
             'tgt_train': './data/de-en-train.en',
             'src_valid': './data/de-en-val.de',
             'tgt_valid': './data/de-en-val.en',
             'name': 'de_en'}

    en_de = {'src_train': './data/de-en-train.en',
             'tgt_train': './data/de-en-train.de',
             'src_valid': './data/de-en-val.en',
             'tgt_valid': './data/de-en-val.de',
             'name': 'en_de'}

    sv_de = {'src_train': './data/de-sv-train.sv',
             'tgt_train': './data/de-sv-train.de',
             'src_valid': './data/de-sv-val.sv',
             'tgt_valid': './data/de-sv-val.de',
             'name': 'sv_de'}

    de_sv = {'src_train': './data/de-sv-train.de',
             'tgt_train': './data/de-sv-train.sv',
             'src_valid': './data/de-sv-val.de',
             'tgt_valid': './data/de-sv-val.sv',
             'name': 'de_sv'}

    en = {'src_train': './data/de-en-train.en',
          'tgt_train': './data/de-en-train.en',
          'src_valid': './data/de-en-val.en',
          'tgt_valid': './data/de-en-val.en',
          'name': 'en'}

    de = {'src_train': './data/de-sv-train.de',
          'tgt_train': './data/de-sv-train.de',
          'src_valid': './data/de-sv-val.de',
          'tgt_valid': './data/de-sv-val.de',
          'name': 'de'}

    sv = {'src_train': './data/de-sv-train.sv',
          'tgt_train': './data/de-sv-train.sv',
          'src_valid': './data/de-sv-val.sv',
          'tgt_valid': './data/de-sv-val.sv',
          'name': 'sv'}

    # zero-shot translation
    sv_en = {'src_train': './data/en-sv-train.sv',
             'tgt_train': './data/en-sv-train.en',
             'src_valid': './data/en-sv-val.sv',
             'tgt_valid': './data/en-sv-val.en',
             'name': 'en_sv'}

    datafiles = [en, de, sv, en_de, de_en, de_sv, sv_de, sv_en]
    


    
class Translator_multi:
    
    def __init__(self, params):
        self.params = params
        
    def train(self):
        
        p = self.params
        
        # Setting a fixed seed for reproducibility.
        torch.manual_seed(p.random_seed)
        random.seed(p.random_seed)
        
        print('Preparing data...', end='')
        sys.stdout.flush()

        self.dataloaders = []
        self.model_names = []
        self.S_vocs = []
        self.T_vocs = []
        self.S_valids = []
        self.T_valids = []

        for ind, dataset in enumerate(p.datafiles):
            self.model_names.append(dataset['name'])
            # Read the source-language data.
            if ind < 3:
                max_train_sentences = p.max_train_sentences_identity
            else:
                max_train_sentences = p.max_train_sentences

            S_train = mt_util.read_lines(dataset['src_train'], max_lines=max_train_sentences)
            S_voc = mt_util.Vocabulary(include_unknown=True, lower=True, max_voc_size=p.src_voc_size)
            S_voc.build(S_train)
            S_valid = mt_util.read_lines(dataset['src_valid'], max_lines=p.max_valid_sentences)

            # Read the target-language data.
            T_train = mt_util.read_lines(dataset['tgt_train'], max_lines=max_train_sentences)
            T_voc = mt_util.Vocabulary(include_unknown=True, lower=True, max_voc_size=p.tgt_voc_size)
            T_voc.build(T_train)
            T_valid = mt_util.read_lines(dataset['tgt_valid'], max_lines=p.max_valid_sentences)
        
            # Batching for the training and validation sets.
            batcher = mt_util.SequenceBatcher(p.device)        
            train_dataset = mt_util.SequenceDataset(S_voc.encode(S_train), T_voc.encode(T_train))
            train_loader = DataLoader(train_dataset, p.batch_size, shuffle=True, collate_fn=batcher)

            self.dataloaders.append(train_loader)
            self.S_vocs.append(S_voc)
            self.T_vocs.append(T_voc)
            self.S_valids.append(S_valid)
            self.T_valids.append(T_valid)

            
        print(' done.')
        
        print('Building model...', end='')
        sys.stdout.flush() 

        # here we will create all three encoder-decoder pairs, the following indecies will be used:
        # 0: en
        # 1: de
        # 2: sv
        self.encoders = []
        self.decoders = []
        for it in range(p.n_languages):

            S_emb = self.S_vocs[it].make_embedding_layer(finetune=True, emb_dim=p.emb_dim)
            T_emb = self.T_vocs[it].make_embedding_layer(finetune=True, emb_dim=p.emb_dim)
        
            # Build the encoder and decoder.
            self.encoders.append(Encoder(S_emb, p))
            self.decoders.append(Decoder(T_emb, p))
        
        inter = Interlingua(p)

        self.models = []
        # en-de
        self.models.append(MultilingualModel(self.encoders[0], inter, self.decoders[0]))
        # de-en
        self.models.append(MultilingualModel(self.encoders[1], inter, self.decoders[1]))
        # sv-de
        self.models.append(MultilingualModel(self.encoders[2], inter, self.decoders[2]))
        # de-sv
        self.models.append(MultilingualModel(self.encoders[0], inter, self.decoders[1]))
        # en-en
        self.models.append(MultilingualModel(self.encoders[1], inter, self.decoders[0]))
        # de-de
        self.models.append(MultilingualModel(self.encoders[1], inter, self.decoders[2]))
        # sv-sv
        self.models.append(MultilingualModel(self.encoders[2], inter, self.decoders[1]))
        # en-sv (zero-shot model, won't be trained directly)
        self.models.append(MultilingualModel(self.encoders[2], inter, self.decoders[0]))
        print(' done.')
                

        self.history = defaultdict(list)
        best_bleu = 0
        for epoch in range(1, p.n_epochs+1):

            for ind, model in enumerate(self.models[:-1]):
                self.ind = ind
                
                self.model = model
                # print(id(model.encoder))
                # print(id(model.interlingua))
                # print(id(model.decoder))
                self.model.to(p.device)
                optimizer = torch.optim.Adam(self.model.parameters(), 
                                             lr=p.learning_rate, weight_decay=p.weight_decay)

                # The loss function is a cross-entropy loss at the token level.
                # We don't include padding tokens when computing the loss.
                loss_func = torch.nn.CrossEntropyLoss(ignore_index=self.T_vocs[ind].get_pad_idx())

                t0 = time.time()

                loss_sum = 0
                for i, (Sbatch, Tbatch) in enumerate(self.dataloaders[ind], 1):
                    
                    # We use teacher forcing to train the decoder.
                    # This means that the input at each decoding step will be the
                    # *gold-standard* word at the previous position.
                    # We create a tensor Tbatch_shifted that contains the previous words.                
                    batch_size, sen_len = Tbatch.shape
                    zero_pad = torch.zeros(batch_size, 1, dtype=torch.long, device=Tbatch.device)
                    Tbatch_shifted = torch.cat([zero_pad, Tbatch[:, :-1]], dim=1)
                    
                    self.model.train()
                    scores = self.model(Sbatch, Tbatch_shifted)

                    loss = loss_func(scores.view(-1, len(self.T_vocs[ind])), Tbatch.view(-1))

                    optimizer.zero_grad()            
                    loss.backward()
                    optimizer.step()
                    loss_sum += loss.item()

                    print('.', end='')
                    sys.stdout.flush()
                    
                    # We periodically print some diagnostics: loss, BLEU score on the validation
                    # set, and the translation of a test sentence.
                    if i % p.n_batches_print == 0:
                        print(f' {i*p.batch_size}') 
                        
                t1 = time.time()
                
                # train_loss = loss_sum / len(self.dataloaders[ind])
                bleu = self.eval_bleu(self.S_valids[ind], self.T_valids[ind])

                self.history[self.model_names[ind]].append(bleu)
                print(f'Epoch {epoch}, model {self.model_names[ind]}: time={t1-t0:.4f}, BLEU={bleu}\n')

                self.model.to('cpu')

            # load sv-en model for evaluation
            ind = -1
            self.model = self.models[ind]
            self.model.to(p.device)
            self.ind = ind

            # evaluate model, store in history and print epoch performance
            bleu = self.eval_bleu(self.S_valids[ind], self.T_valids[ind])
            self.history[self.model_names[ind]].append(bleu)
            print(f'Epoch {epoch} results for {self.model_names[ind]}:\n\t BLEU={bleu:.4f}\n')
            self.model.to('cpu')
        
            # Plot history
            plt.figure(figsize=(6, 5))
            plt.style.use('seaborn')
            x = range(1, epoch+1)
            # lines
            for mn in self.model_names:
                plt.plot(x, self.history[mn], label=mn)
            plt.legend()
            # text
            plt.suptitle('Multilingual model', fontsize=20)
            plt.ylabel('BLEU', fontsize=14)
            plt.xlabel('epoch', fontsize=14)
            # save
            plt.savefig('fig.png', dpi=150)
            plt.clf()

            if bleu > best_bleu:
                names = ['en', 'de', 'sv']
                for encoder, decoder, n in zip(self.encoders, self.decoders, names):
                    torch.save(encoder.state_dict(), f"encoder.{n}")
                    torch.save(decoder.state_dict(), f"decoder.{n}")
                torch.save(inter.state_dict(), f"inter")

    # Utility function that runs the encoder and decoder for a batch.
    # We then map the target-language word indices to the corresponding strings,
    # and cut off the sentences after the end-of-sentence marker.
    # For visualization, we'll also return all the attention scores.
    def translate_batch(self, src):
        with torch.no_grad():
            encoded = self.S_vocs[self.ind].encode(src)
            max_len = max(len(s) for s in encoded)
            pad = self.S_vocs[self.ind].get_pad_idx()
            for s in encoded:
                s.extend([pad]*(max_len-len(s)))
            enc_tensor = torch.tensor(encoded, device=self.params.device)
            tgt, attn = self.model.greedy_decode(enc_tensor, max_len=2*max_len)
            tgt_dec = self.T_vocs[self.ind].decode(tgt[:,1:].cpu().numpy())
            for s in tgt_dec:
                try:
                    eos_ix = s.index(mt_util.EOS)
                    del s[eos_ix:]
                except:
                    pass
            return tgt_dec, attn.cpu().numpy()
        
    # Translates a set of sentences in the source language.
    def translate(self, src, batch_size=512):
        self.model.eval()
        out = []
        batch = []
        for sen in src:
            batch.append(sen)
            if len(batch) == batch_size:
                translated_batch, _ = self.translate_batch(batch)
                out.extend(translated_batch)
                batch = []
        if len(batch) > 0:
            translated_batch, _ = self.translate_batch(batch)
            out.extend(translated_batch)
        return out
         
    # Translates a single sentence and returns the translation and attention scores.
    def translate_with_attention(self, sentence):
        translated, attn = self.translate_batch([sentence])
        attn = attn.squeeze(1)
        return translated[0], attn
        
    # Runs the translator, and then uses the SacreBLEU evaluator to compute a BLEU score.
    def eval_bleu(self, src, ref):
        translated = self.translate(src)
        translated = [' '.join(s) for s in translated]
        ref = [' '.join(s).lower() for s in ref]
        return sacrebleu.raw_corpus_bleu(translated, [ref], 0.01).score
        


In [ ]:
translator_multi = Translator_multi(TranslatorParameters_multi())
translator_multi.train()
